In [2]:
import pandas as pd

from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator

### Data and Model Preparation

In [3]:
# Data Loading
data_dir = './_data/'
train_filenames = ['2022-04-22 02:07:18_0_cont.csv']
test_filenames = ['2022-04-22 02:07:18_0_cont.csv']

# Prepare Train Data
df_l = []
for f in train_filenames:
    filepath = data_dir + f
    df_l.append(pd.read_csv(filepath, index_col=None, header=0))
df_train = pd.concat(df_l, axis=0, ignore_index=True)

# Prepare Test Data
df_l = []
for f in test_filenames:
    filepath = data_dir + f
    df_l.append(pd.read_csv(filepath, index_col=None, header=0))
df_test = pd.concat(df_l, axis=0, ignore_index=True)

In [4]:
# Construct PGM
model_struct = BayesianNetwork([('ball_state','keypress'), ('paddle_state', 'keypress')])


### Train

In [5]:
# Training using MLE
mle = MaximumLikelihoodEstimator(model=model_struct, data=df_train)

In [6]:
# Print Learned Parameters
print(mle.estimate_cpd(node="keypress"))

+--------------+--------------------+-----+------------------------+
| ball_state   | ball_state(left)   | ... | ball_state(stagnant)   |
+--------------+--------------------+-----+------------------------+
| paddle_state | paddle_state(left) | ... | paddle_state(stagnant) |
+--------------+--------------------+-----+------------------------+
| keypress(0)  | 0.0                | ... | 0.9394812680115274     |
+--------------+--------------------+-----+------------------------+
| keypress(1)  | 0.0                | ... | 0.040345821325648415   |
+--------------+--------------------+-----+------------------------+
| keypress(2)  | 0.0                | ... | 0.01440922190201729    |
+--------------+--------------------+-----+------------------------+
| keypress(3)  | 1.0                | ... | 0.005763688760806916   |
+--------------+--------------------+-----+------------------------+


### Test

In [7]:
# Helper Functions
def test_state_generation(ball_state, paddle_state):
    if (ball_state+ ',' + paddle_state) == 'left,left': 
        state1 = 0
        state2 = 0
    if (ball_state+ ',' + paddle_state) == 'left,right': 
        state1 = 0
        state2 = 1
    if (ball_state+ ',' + paddle_state) == 'left,stagnant':
        state1 = 0
        state2 = 2
    if (ball_state+ ',' + paddle_state) == 'right,left': 
        state1 = 1
        state2 = 0 
    if (ball_state+ ',' + paddle_state) == 'right,right': 
        state1 = 1
        state2 = 1
    if (ball_state+ ',' + paddle_state) == 'right,stagnant': 
        state1 = 1
        state2 = 2
    if (ball_state+ ',' + paddle_state) == 'stagnant,left': 
        state1 = 2
        state2 = 0
    if (ball_state+ ',' + paddle_state) == 'stagnant,right': 
        state1 = 2
        state2 = 1
    if (ball_state+ ',' + paddle_state) == 'stagnant,stagnant': 
        state1 = 2
        state2 = 2
    return state1, state2

In [11]:
# Test Setup
pos = 0
neg = 0
model_data = mle.estimate_cpd(node="keypress").values

# Main Test Loop
for i, row in df_test.iterrows():
    ball_state = row['ball_state']
    paddle_state = row['paddle_state']
    keypress = row['keypress']

    state1, state2 = test_state_generation(ball_state, paddle_state)

    choices = [model_data[i][state1][state2] for i in range (4)]
    best_choice = max(choices)

    predicted = choices.index(best_choice)
    if predicted == 2: predicted = 3
    if predicted == 3: predicted = 4  

    if predicted == keypress: 
        pos += 1
    else: neg += 1

acc = pos/(pos+neg)
print(f'Accuracy : {acc}')
print(f'Pos      : {pos}')
print(f'Neg      : {neg}')

Accuracy : 0.8627450980392157
Pos      : 572
Neg      : 91
